In [21]:
import numpy as np
import pandas as pd
from numba import njit 
from collections import namedtuple
import random
import math
from collections import namedtuple

Point = namedtuple("Point", ['x', 'y'])

In [ ]:
a=[1,2,3]

@njit(fastmath=True) 
def numba_test(a):
    for i in range(len(a)):
        a[i]= a[i]*2
    return a.pop()
        
numba_test(a)

In [42]:
a=[1,2,3]

@njit(fastmath=True) 
def numba_test(a):
    m=[]
    while len(a)!=0:
        k=a.pop()*2
        m.append(k)
    return m
        
numba_test(a)

[6, 4, 2]

In [2]:
np.random.exponential(1000)

375.41104428850156

In [22]:
with open('tsp_5_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])

    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))

## greedy search

In [23]:
def length(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
##a very pool greedy search
solution_points=[points[i-1]]
solution_index=[0]
available_points=list(range(1,nodeCount))

for position in range(1,nodeCount):
    better_point= available_points[0]
    dis= length(solution_points[position-1], points[better_point])
    for k in available_points:
        if length(solution_points[position-1], points[k])< dis:
            better_point= k
            dis=length(solution_points[position-1], points[k])
    solution_index.append(better_point)
    solution_points.append(points[better_point])
    available_points.pop((better_point-1))


solution_points
solution_index    
solution=solution_index           
obj = length(points[solution[-1]], points[solution[0]])
for index in range(0, nodeCount-1):
    obj += length(points[solution[index]], points[solution[index+1]])

solution

[0, 4, 3, 2, 1]

## 2-opt local search

In [24]:
##regular swap using list, instead of np.array
def swap_dist(index_1, index_2, solution):
    l= len(solution)
    curr_dist=length(points[solution[((index_1-1)%l)]], points[solution[index_1]]) + length(points[solution[((index_2+1)%l)]], points[solution[index_2]])
    swap_dist= length(points[solution[((index_1-1)%l)]], points[solution[index_2]]) + length(points[solution[((index_2+1)%l)]], points[solution[index_1]])
    return swap_dist-curr_dist


solution=list(range(nodeCount))
for i in range(10):
    for position in range(1,nodeCount):
        curr_dist=0
        best_point = solution[position]
        original_point= solution[position]
        best_index=position
        for index in range(position+1, nodeCount):
            if swap_dist(position, index, solution)<=curr_dist:
                curr_dist= swap_dist(position, index, solution)
                best_point= solution[index]
                best_index=index
        solution[position] = best_point
        solution[best_index] = original_point

# calculate the length of the tour
obj = length(points[solution[-1]], points[solution[0]])
for index in range(0, nodeCount-1):
    obj += length(points[solution[index]], points[solution[index+1]])

solution

[0, 1, 2, 3, 4]

## 2-opt search using numba

In [25]:
with open('tsp_5_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])

#     points = []
#     for i in range(1, nodeCount+1):
#         line = lines[i]
#         parts = line.split()
#         points.append(Point(float(parts[0]), float(parts[1])))

    points_np=np.zeros((nodeCount,2))
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points_np[i-1][0]= float(parts[0])
        points_np[i-1][1]= float(parts[1])
    
    solution=np.zeros(nodeCount)
    for i in range(nodeCount):
        solution[i]=i

In [26]:
import numpy as np
from numba import njit 

@njit(fastmath=True) 
def numba_swap(points_np, solution,nodeCount):
    
    ## 2-OPT swap local search:
    for position in range(1,nodeCount):
        curr_dist=0
        best_point = solution[position]
        original_point= solution[position]
        best_index=position
        for index in range(position+1, nodeCount):
            if swap_dist(position, index, solution)<=curr_dist:
                curr_dist= swap_dist(position, index, solution)
                best_point= solution[index]
                best_index=index
        solution[position] = best_point
        solution[best_index] = original_point
    
    return solution

@njit(fastmath=True) 
def swap_dist(index_1, index_2, solution):
    l= len(solution)
    curr_dist=length(points_np[int(solution[((index_1-1)%l)])], points_np[int(solution[index_1])]) + length(points_np[int(solution[((index_2+1)%l)])], points_np[int(solution[index_2])]) 
    swap_dist= length(points_np[int(solution[((index_1-1)%l)])], points_np[int(solution[index_2])])  + length(points_np[int(solution[((index_2+1)%l)])], points_np[int(solution[index_1])])
    return swap_dist-curr_dist
    
@njit(fastmath=True) 
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [27]:
for n in range(10):
    solution= numba_swap(points_np, solution,nodeCount)
    solution=solution.astype(int)
solution

array([0, 1, 2, 3, 4])

## 2-opt search using numba and simulated annealing

In [36]:
with open(r'data/tsp_51_1.txt') as input_data:
#     lines = input_data.split('\n')
    lines= list(input_data)

    nodeCount = int(lines[0])
    
    
    points = []
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points.append(Point(float(parts[0]), float(parts[1])))
    
    
    points_np=np.zeros((nodeCount,2))
    for i in range(1, nodeCount+1):
        line = lines[i]
        parts = line.split()
        points_np[i-1][0]= float(parts[0])
        points_np[i-1][1]= float(parts[1])
    
    solution=np.zeros(nodeCount)
    for i in range(nodeCount):
        solution[i]=i

In [37]:
def length_obj(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)



@njit(fastmath=True) 
def numba_swap(points_np, solution,nodeCount, t=0.000000001):

    ## 2-OPT swap local search:
    for position in range(1,nodeCount):
#         curr_dist=0
#         best_point = solution[position]
#         original_point= solution[position]
#         best_index=position
        for index in range(1, nodeCount):
            orginal_point= int(solution[position])
            if swap_dist(position, index, solution, points_np)<=0:
#                 curr_dist= swap_dist(position, index, solution, points_np)
#                 best_point= solution[index]
#                 best_index=index
                solution[position]= int(solution[index])
                solution[index] = original_point
                
        
#             elif np.exp(-(swap_dist(position, index, solution, points_np)- curr_dist)/t)>=random.random():
#                 best_point= solution[index]
#                 best_index=index
# #                 m=solution[index]
# #                 solution[position]= m
# #                 solution[index] = original_point
                
#         solution[position] = best_point
#         solution[best_index] = original_point
    
    return solution


@njit(fastmath=True) 
def swap_dist(index_1, index_2, solution, points_np):
    l= len(solution)
    index_low= min(index_1, index_2)
    index_high=max(index_1, index_2)
    if index_high-index_low==1:
        curr_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_low])])+ length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_high])]))
        swap_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_high])])+length(points_np[int(solution[((index_high+1)%l)])],points_np[int(solution[index_low])]))

    else:
        curr_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_low])])+length(points_np[int(solution[((index_low+1)%l)])], points_np[int(solution[index_low])])
        + length(points_np[int(solution[((index_high-1)%l)])], points_np[int(solution[index_high])])+ length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_high])]))
        swap_dist= (length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_high])]) + length(points_np[int(solution[((index_low+1)%l)])], points_np[int(solution[index_high])])
        + length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_low])]) + length(points_np[int(solution[((index_high-1)%l)])], points_np[int(solution[index_low])]))
    return swap_dist-curr_dist
        
@njit(fastmath=True) 
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [201]:
length(points_np[0], points_np[2])

1.0

In [ ]:
0 1 2 3 4  ;   0 4 2 3 1    01+12+23+34   03+32 +21+ 14

In [ ]:
1 and 2 : 01+12+21+23 02+21+12+13

In [ ]:
0 1 2 3 4 5: 1 and 4: 01+12+34+35  04+42+13+15

In [213]:
solution=[0,3,2,1,4]

In [214]:
swap_dist(2, 3, solution, points_np)

curr 2.118033988749895
0.0
swap 2.53224755112299


0.4142135623730949

In [218]:
solution

array([0., 1., 2., 3., 4.])

In [210]:
best_solution=[0,3,2,1,4]
obj = length(points_np[int(best_solution[-1])], points_np[int(best_solution[0])])
for index in range(0, nodeCount-1):
    obj += length(points_np[int(best_solution[index])], points_np[int(best_solution[index+1])])
obj


5.03224755112299

In [211]:
best_solution=[0,3,1,2,4]
obj = length(points_np[int(best_solution[-1])], points_np[int(best_solution[0])])
for index in range(0, nodeCount-1):
    obj += length(points_np[int(best_solution[index])], points_np[int(best_solution[index+1])])
obj


5.446461113496085

In [38]:
best_obj=10000000000000000
best_solution=[]
for i in range(10):
    for k in range(100):
        solution= numba_swap(points_np, solution,nodeCount,(i*8+1)/80)
        solution=solution.astype(int)
        obj = length_obj(points[solution[-1]], points[solution[0]])
        for index in range(0, nodeCount-1):
            obj += length_obj(points[solution[index]], points[solution[index+1]])

        if obj<=best_obj:
            best_solution=[]
            for point_index in solution:
                best_solution.append(point_index)
# calculate the length of the tour
obj = length(points_np[int(best_solution[-1])], points_np[int(best_solution[0])])
for index in range(0, nodeCount-1):
    obj += length(points_np[int(best_solution[index])], points_np[int(best_solution[index+1])])


In [39]:
best_solution

[0,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [270]:
obj

4.0

In [223]:
best_solution

[0,
 49,
 50,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4]

In [109]:
obj

175.3323831946298

In [75]:
best_obj=10000000000000000
best_solution=[]
for i in range(10):
    for k in range(3):
        solution= numba_swap(points_np, solution,nodeCount, (i*8+1)/100)
        solution=solution.astype(int)
        obj = length_obj(points[solution[-1]], points[solution[0]])
        for index in range(0, nodeCount-1):
            obj += length_obj(points[solution[index]], points[solution[index+1]])

        if obj<=best_obj:
            best_solution=[]
            for point_index in solution:
                best_solution.append(point_index)

n=0
while n>3:
    solution= numba_swap(points_np, solution,nodeCount)
    solution=solution.astype(int)
    obj = length_obj(points[solution[-1]], points[solution[0]])
    for index in range(0, nodeCount-1):
        obj += length_obj(points[solution[index]], points[solution[index+1]])

    if obj<=best_obj:
        best_solution=[]
        for point_index in solution:
            best_solution.append(point_index)
    else:
        n+=1

solution= best_solution
# calculate the length of the tour
obj = length(points_np[int(solution[-1])], points_np[int(solution[0])])
for index in range(0, nodeCount-1):
    obj += length(points_np[int(solution[index])], points_np[int(solution[index+1])])


In [76]:
obj

573.9837201018063

In [56]:
solution[:5]

[0, 33, 47, 26, 6]

In [18]:
###previous method saved:
def length_obj(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)


@njit(fastmath=True) 
def numba_swap(points_np, solution,nodeCount, t=0.000000001):

    ## 2-OPT swap local search:
    for position in range(1,nodeCount):
        curr_dist=0
#         best_point = solution[position]
#         original_point= solution[position]
#         best_index=position
        for index in range(position+1, nodeCount):
            orginal_point= solution[position]
            if swap_dist(position, index, solution, points_np)<=curr_dist:
                curr_dist= swap_dist(position, index, solution, points_np)
#                 best_point= solution[index]
#                 best_index=index
                solution[position]= solution[index]
                solution[index] = original_point

                
            

#             elif np.exp(-(swap_dist(position, index, solution, points_np)- curr_dist)/t)>=random.random():
#                 best_point= solution[index]
#                 best_index=index
# #                 m=solution[index]
# #                 solution[position]= m
# #                 solution[index] = original_point
                
#         solution[position] = best_point
#         solution[best_index] = original_point
    
    return solution


@njit(fastmath=True) 
def swap_dist(index_1, index_2, solution, points_np):
    l= len(solution)
    index_low= min(index_1, index_2)
    index_high=max(index_1, index_2)
    if index_high-index_low==1:
        curr_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_low])])+ length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_high])]))
        swap_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_high])])+length(points_np[int(solution[((index_high+1)%l)])],points_np[int(solution[index_low])]))

    else:
        curr_dist=(length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_low])])+length(points_np[int(solution[((index_low+1)%l)])], points_np[int(solution[index_low])])
        + length(points_np[int(solution[((index_high-1)%l)])], points_np[int(solution[index_high])])+ length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_high])]))
        swap_dist= (length(points_np[int(solution[((index_low-1)%l)])], points_np[int(solution[index_high])]) + length(points_np[int(solution[((index_low+1)%l)])], points_np[int(solution[index_high])])
        + length(points_np[int(solution[((index_high+1)%l)])], points_np[int(solution[index_low])]) + length(points_np[int(solution[((index_high-1)%l)])], points_np[int(solution[index_low])]))
    return swap_dist-curr_dist
        
@njit(fastmath=True) 
def length(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

## K-opt local search with random start and SA

## MIP